In [1]:
import time
import statistics
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import mode

In [2]:
#Load the image in black and white (0 - b/w, 1 - color).
def inner_box(img,count,height):
    
    import operator
    import os

    #Invert the image to be white on black for compatibility with findContours function.
    imgray = 255 - img

    # rects1 imp
    kernel = np.ones((2,5),np.uint8)
    imgray1 = cv2.dilate(imgray,kernel,iterations = 1)
    kernel = np.ones((5,2),np.uint8)
    imgray1 = cv2.dilate(imgray1,kernel,iterations = 1)
    thresh1 = cv2.adaptiveThreshold(imgray1, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects1 = [cv2.boundingRect(cnt) for cnt in contours]

    # rects2 imp
    imgray2 = cv2.blur(imgray,(10,3))
    thresh2 = cv2.adaptiveThreshold(imgray2, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects2 = [cv2.boundingRect(cnt) for cnt in contours]

    # rects3 imp
    imgray3 = cv2.blur(imgray,(3,10))
    thresh3 = cv2.adaptiveThreshold(imgray3, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh3, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects3 = [cv2.boundingRect(cnt) for cnt in contours]

    #rects4 imp
    kernel = np.ones((2,7),np.uint8)
    imgray4 = cv2.dilate(imgray,kernel,iterations = 1)
    thresh4 = cv2.adaptiveThreshold(imgray4, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh4, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects4 = [cv2.boundingRect(cnt) for cnt in contours]

    
    #rects5 imp
    kernel = np.ones((2,7),np.uint8)
    imgray5 = cv2.dilate(imgray,kernel,iterations = 1)
    thresh5 = cv2.adaptiveThreshold(imgray5, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh5, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects5 = [cv2.boundingRect(cnt) for cnt in contours]

    # Checking if Student ID box or not and taking unique rectangles only
    if count!=0:
        rects=[]
        for i in rects1:
            if (i[2]>height/75 and i[2]<height/50 and i[3]>height/75 and i[3]<height/50 and i not in rects):
                rects.append(i)

        for i in rects2:
            if (i[2]>height/75 and i[2]<height/50 and i[3]>height/75 and i[3]<height/50 and i not in rects):
                rects.append(i)

        for i in rects3:
            if (i[2]>height/75 and i[2]<height/50 and i[3]>height/75 and i[3]<height/50 and i not in rects):
                rects.append(i)

        for i in rects4:
            if (i[2]>height/75 and i[2]<height/50 and i[3]>height/75 and i[3]<height/50 and i not in rects):
                rects.append(i)

        for i in rects5:
            if (i[2]>height/75 and i[2]<height/50 and i[3]>height/75 and i[3]<height/50 and i not in rects):
                rects.append(i)
    else:
        rects=[]
        for i in rects1:
            if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
                rects.append(i)

        for i in rects2:
            if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
                rects.append(i)

        for i in rects3:
            if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
                rects.append(i)

        for i in rects4:
            if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
                rects.append(i)

        for i in rects5:
            if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
                rects.append(i)


   # Removing similar rectangales
    rects_final=rects.copy()
    for i in range(0,len(rects)-1):
        try:
            for j in range(i+1,len(rects)):
                if (abs(rects[i][0] - rects[j][0]) < 10 and abs(rects[i][1] - rects[j][1]) < 10):
                    if (rects[i][2] < rects[j][2] and rects[i][3] < rects[j][3]):
                        rects_final.remove(rects[j])
                    else:
                        rects_final.remove(rects[i])
        except:
            continue
    print(len(rects_final))
    rects=rects_final.copy()

    #Calculate the combined bounding rectangle points.
    top_x = min([x for (x, y, w, h) in rects])
    top_y = min([y for (x, y, w, h) in rects])
    bottom_x = max([x+w for (x, y, w, h) in rects])
    bottom_y = max([y+h for (x, y, w, h) in rects])

    # Calculating number of approx rows in the box
    h_mode = mode([h for (x, y, w, h) in rects])[0]
    n=int(((bottom_y-top_y)/h_mode))

    temp=[]
    b = [[] for i in range(0, n+3)]
    for j in range (1,n+3):
        for i in rects:
            x=i[0]
            y=i[1]
            w=i[2]
            h=i[3]
            if ((y > bottom_y-h_mode*j-h_mode/2.2) and (i not in temp)):
                b[j].append(i)
                temp.append(i)
            b[j]=sorted(b[j],key=lambda x:(-x[0]))
    b.pop(0)
    
    tar_dir=r"C:\Users\Rahul\Desktop\Capstone\form\\"+str(count)+"\\"
    if not os.path.exists(tar_dir):
        print("here2")
        os.makedirs(tar_dir) 



    count=0
    for i in b:
        len1=len(i)
        for j in i:
            try:
                x=j[0]
                y=j[1]
                w=j[2]
                h=j[3]
                #if ((w>40 and w<70) and (h>40 and h <70)):
                cv2.imwrite(tar_dir+str(count)+".jpg",img[y:y+h,x:x+w])
                time.sleep(0.01)
                count+=1
            except:
                continue

In [9]:
#Load the image in black and white (0 - b/w, 1 - color).
img = cv2.imread(r'C:\Users\Rahul\Desktop\Capstone\praxis3.jpg', 0)

#Get the height and width of the image.
height, width = img.shape[:2]

#Invert the image to be white on black for compatibility with findContours function.
imgray = 255 - img

# Remove noise using dilate and thershold
thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,2)
kernel = np.ones((2,2),np.uint8)
thresh = cv2.dilate(thresh,kernel,iterations = 1)

#Find all the contours in thresh. 
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

#Find bounding rectangles for each contour.
rects = [cv2.boundingRect(cnt) for cnt in contours]
rects2=[]
count=0
for i in rects:
    # hierarchy [Next, Previous, First_Child, Parent]
    if (hierarchy[0][count][2]>0 and hierarchy[0][count][3]<0):   # condn that contour is parent and have child.
        if (i[2]>thresh.shape[1]/3 and i[2] < thresh.shape[1]):   # condn on width of contours
            rects2.append(i)
    count+=1
h_mode = mode([h for (x, y, w, h) in rects2])[0]
#print(h_mode1)
# to remove noisy contour i.e. which are small in height if these
if h_mode < height/20:
    rects2=[]
    count=0
    for i in rects:
        if (hierarchy[0][count][2]>0):
            if (i[2]>thresh.shape[1]/3 and i[2] < thresh.shape[1] and i[3] > h_mode*1.2 ):
                rects2.append(i)
        count+=1
#print(len(rects2))

# to remove duplicate contours having very less coordinate difference
rects3=rects2.copy()
for i in range(0,len(rects2)-1):
    try:
        for j in range(i+1,len(rects2)):
            if (rects2[i][1] - rects2[j][1] <50):
                if rects2[i][2]>rects2[j][2]:
                    value=rects2[j]
                    rects3.remove(value)
                else :
                    value=rects2[i]
                    rects3.remove(value)
    except:
        continue
#print(len(rects3))

# sort in correct order
rects3=sorted(rects3,key=lambda x:(x[1]))

# saving all
count=0

out_rects=[]
for j in rects3:
    try:
        x=j[0]
        y=j[1]
        w=j[2]
        h=j[3]
        inner_box(img[y:y+h,x:x+w],count,height)
        time.sleep(0.001)
        count+=1
    except:
        continue

6
here2
116
here2
30
here2
58
here2
